In [ ]:
!apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (17.3 MB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 131015 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-e

In [ ]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
import pytesseract
import os

def login_with_captcha(base_url, headers, username, password):
    response = requests.get(base_url, headers=headers)

    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')

        captcha_img_tag = soup.find('img', id='imgCaptcha')

        if captcha_img_tag:
            captcha_img_url = base_url + "/" + captcha_img_tag['src']

            captcha_response = requests.get(captcha_img_url, headers=headers)

            if captcha_response.status_code == 200:
                with open('captcha_image.jpeg', 'wb') as f:
                    f.write(captcha_response.content)

                img = Image.open('captcha_image.jpeg')
                captcha_text = pytesseract.image_to_string(img)
                captcha_text = captcha_text.strip()

                os.remove('captcha_image.jpeg')

                form_data = {
                    'ScriptManager1': 'updpnl|btnLogin',
                    '__EVENTTARGET': '',
                    '__EVENTARGUMENT': '',
                    '__VIEWSTATE': soup.find('input', {'name': '__VIEWSTATE'})['value'],
                    '__VIEWSTATEGENERATOR': soup.find('input', {'name': '__VIEWSTATEGENERATOR'})['value'],
                    '__EVENTVALIDATION': soup.find('input', {'name': '__EVENTVALIDATION'})['value'],
                    'txtUserid': username,
                    'txtpassword': password,
                    'txtCaptcha': captcha_text,
                    '__ASYNCPOST': 'true',
                    'btnLogin': 'Sign in'
                }

                post_response = requests.post(base_url, data=form_data, headers=headers)

                if post_response.status_code == 200:
                    if 'Invalid Captcha' in post_response.text:
                        print('Captcha Error')
                    else:

                        student_home_url = "/studenthomepage.aspx"
                        full_student_home_url = base_url + student_home_url
                        student_home_response = requests.get(full_student_home_url, headers=headers)

                        if student_home_response.status_code != 200:
                            print('Failed to redirect to Student Home Page')
                            return

                else:
                    print("Login request failed. Status code:", post_response.status_code)
            else:
                print("Failed to download CAPTCHA image. Status code:", captcha_response.status_code)
        else:
            print("CAPTCHA image not found on the page.")
    else:
        print("Failed to fetch login page. Status code:", response.status_code)


def get_marks(Sem):

    student_academics_url = "/Academics.aspx"
    full_academic_url = base_url + student_academics_url
    student_academic_response = requests.get(full_academic_url, headers=headers)

    if student_academic_response.status_code != 200:
            print('Redirected to Academics Page successfully')
            return

    with open('output.txt', 'w') as file:
        file.write(student_academic_response.text)

    soup = BeautifulSoup(student_academic_response.text, 'html.parser')
    div_panel_group = soup.find('div', class_='panel-group internalMarks')
    print('-------------------------------------------------------------------------')
    if div_panel_group:
        lines = (str(div_panel_group)).split('\n')
        subject_details = []
        for line in lines:
            if 'Subject Code' in line:
                subject_details.append(line.split('</b>')[1])

        for i in range(9):
            print(subject_details[i])

            marks_obtained_id = f'ContentPlaceHolder1_RepeaterPrintInternal_lblTotal_{i}'
            marks_obtained_element = div_panel_group.find('span', id=marks_obtained_id)
            if marks_obtained_element:
                marks_obtained = marks_obtained_element.text.strip().split(':')[-1].strip()
                print("Marks Obtained:", marks_obtained)

            max_marks_id = f'ContentPlaceHolder1_RepeaterPrintInternal_Label1_{i}'
            max_marks_element = div_panel_group.find('span', id=max_marks_id)
            if max_marks_element:
                max_marks = max_marks_element.text.strip().split(':')[-1].strip()
                print("Max Marks:", max_marks)

            assignment_table_id = f'ContentPlaceHolder1_RepeaterPrintInternal_pnlAssignment_{i}'
            assignment_table_element = div_panel_group.find('div', id=assignment_table_id)

            if assignment_table_element:
                assignment_table = assignment_table_element.find('table')
            else:
                assignment_table = None

            if assignment_table:
                rows = assignment_table.find_all('tr')
                print("Assignment Details:")
                for row in rows[1:]:
                    columns = row.find_all('td')
                    assignment_name = columns[0].text.strip()
                    max_marks = columns[1].text.strip()
                    marks_obtained = columns[2].text.strip()
                    print(f"Assignment: {assignment_name}, Max Marks: {max_marks}, Marks Obtained: {marks_obtained}")

            print('-------------------------------------------------------------------------')

    else:
        print("Div element with class 'panel-group internalMarks' not found in the HTML content.")


base_url = "https://slcm.manipal.edu"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'Cookie': 'ASP.NET_SessionId=junptmnf0ncgtk4wrx4qonpm',
    'Referer': 'https://slcm.manipal.edu',
}
username = 'Enter_your_SLCM_username'
password = 'Enter_your_SLCM_username'

login_with_captcha(base_url, headers, username, password)
get_marks(1)


In [ ]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
import pytesseract
import os

def login_with_captcha(base_url, headers, username, password):
    response = requests.get(base_url, headers=headers)

    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')

        captcha_img_tag = soup.find('img', id='imgCaptcha')

        if captcha_img_tag:
            captcha_img_url = base_url + "/" + captcha_img_tag['src']

            captcha_response = requests.get(captcha_img_url, headers=headers)

            if captcha_response.status_code == 200:
                with open('captcha_image.jpeg', 'wb') as f:
                    f.write(captcha_response.content)

                img = Image.open('captcha_image.jpeg')
                captcha_text = pytesseract.image_to_string(img)
                captcha_text = captcha_text.strip()

                os.remove('captcha_image.jpeg')

                form_data = {
                    'ScriptManager1': 'updpnl|btnLogin',
                    '__EVENTTARGET': '',
                    '__EVENTARGUMENT': '',
                    '__VIEWSTATE': soup.find('input', {'name': '__VIEWSTATE'})['value'],
                    '__VIEWSTATEGENERATOR': soup.find('input', {'name': '__VIEWSTATEGENERATOR'})['value'],
                    '__EVENTVALIDATION': soup.find('input', {'name': '__EVENTVALIDATION'})['value'],
                    'txtUserid': username,
                    'txtpassword': password,
                    'txtCaptcha': captcha_text,
                    '__ASYNCPOST': 'true',
                    'btnLogin': 'Sign in'
                }

                post_response = requests.post(base_url, data=form_data, headers=headers)

                if post_response.status_code == 200:
                    if 'Invalid Captcha' in post_response.text:
                        print('Captcha Error')
                    else:

                        student_home_url = "/studenthomepage.aspx"
                        full_student_home_url = base_url + student_home_url
                        student_home_response = requests.get(full_student_home_url, headers=headers)

                        if student_home_response.status_code != 200:
                            print('Failed to redirect to Student Home Page')
                            return

                else:
                    print("Login request failed. Status code:", post_response.status_code)
            else:
                print("Failed to download CAPTCHA image. Status code:", captcha_response.status_code)
        else:
            print("CAPTCHA image not found on the page.")
    else:
        print("Failed to fetch login page. Status code:", response.status_code)


def get_marks(Sem):

    student_academics_url = "/Academics.aspx"
    full_academic_url = base_url + student_academics_url
    student_academic_response = requests.get(full_academic_url, headers=headers)

    if student_academic_response.status_code != 200:
            print('Redirected to Academics Page successfully')
            return

    with open('output.txt', 'w') as file:
        file.write(student_academic_response.text)

    soup = BeautifulSoup(student_academic_response.text, 'html.parser')
    div_panel_group = soup.find('div', class_='panel-group internalMarks')

    marks_data = []

    if div_panel_group:
        lines = str(div_panel_group).split('\n')
        subject_details = []
        for line in lines:
            if 'Subject Code' in line:
                subject_details.append(line.split('</b>')[1])

        for i in range(9):
            marks_text = subject_details[i] + "\n"
            marks_text += "--------------------------------\n"

            marks_obtained_id = f'ContentPlaceHolder1_RepeaterPrintInternal_lblTotal_{i}'
            marks_obtained_element = div_panel_group.find('span', id=marks_obtained_id)
            if marks_obtained_element:
                marks_obtained = marks_obtained_element.text.strip().split(':')[-1].strip()
                marks_text += "Marks Obtained: " + marks_obtained + "\n"

            max_marks_id = f'ContentPlaceHolder1_RepeaterPrintInternal_Label1_{i}'
            max_marks_element = div_panel_group.find('span', id=max_marks_id)
            if max_marks_element:
                max_marks = max_marks_element.text.strip().split(':')[-1].strip()
                marks_text += "Max Marks: " + max_marks + "\n"

            assignment_table_id = f'ContentPlaceHolder1_RepeaterPrintInternal_pnlAssignment_{i}'
            assignment_table_element = div_panel_group.find('div', id=assignment_table_id)

            if assignment_table_element:
                marks_text += "Assignment Details:\n"
                assignment_table = assignment_table_element.find('table')
                rows = assignment_table.find_all('tr')
                for row in rows[1:]:
                    columns = row.find_all('td')
                    assignment_name = columns[0].text.strip()
                    max_marks = columns[1].text.strip()
                    marks_obtained = columns[2].text.strip()
                    marks_text += f"Assignment: {assignment_name}, Max Marks: {max_marks}, Marks Obtained: {marks_obtained}\n"

            marks_text += "--------------------------------\n\n"
            marks_data.append(marks_text)

    else:
        print("Div element with class 'panel-group internalMarks' not found in the HTML content.")

    return marks_data


base_url = "https://slcm.manipal.edu"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'Cookie': 'ASP.NET_SessionId=junptmnf0ncgtk4wrx4qonpm',
    'Referer': 'https://slcm.manipal.edu',
}
username = 'Enter_your_SLCM_username'
password = 'Enter_your_SLCM_password'

login_with_captcha(base_url, headers, username, password)
marks_data = get_marks(1)

# Writing marks data to a text file
with open('Marks_6_sem.txt', 'w') as file:
    file.writelines(marks_data)


In [ ]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
import pytesseract
import os

def login_with_captcha(base_url, headers, username, password):
    response = requests.get(base_url, headers=headers)

    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')

        captcha_img_tag = soup.find('img', id='imgCaptcha')

        if captcha_img_tag:
            captcha_img_url = base_url + "/" + captcha_img_tag['src']

            captcha_response = requests.get(captcha_img_url, headers=headers)

            if captcha_response.status_code == 200:
                with open('captcha_image.jpeg', 'wb') as f:
                    f.write(captcha_response.content)

                img = Image.open('captcha_image.jpeg')
                captcha_text = pytesseract.image_to_string(img)
                captcha_text = captcha_text.strip()

                os.remove('captcha_image.jpeg')

                form_data = {
                    'ScriptManager1': 'updpnl|btnLogin',
                    '__EVENTTARGET': '',
                    '__EVENTARGUMENT': '',
                    '__VIEWSTATE': soup.find('input', {'name': '__VIEWSTATE'})['value'],
                    '__VIEWSTATEGENERATOR': soup.find('input', {'name': '__VIEWSTATEGENERATOR'})['value'],
                    '__EVENTVALIDATION': soup.find('input', {'name': '__EVENTVALIDATION'})['value'],
                    'txtUserid': username,
                    'txtpassword': password,
                    'txtCaptcha': captcha_text,
                    '__ASYNCPOST': 'true',
                    'btnLogin': 'Sign in'
                }

                post_response = requests.post(base_url, data=form_data, headers=headers)

                if post_response.status_code == 200:
                    if 'Invalid Captcha' in post_response.text:
                        print('Captcha Error')
                    else:

                        student_home_url = "/studenthomepage.aspx"
                        full_student_home_url = base_url + student_home_url
                        student_home_response = requests.get(full_student_home_url, headers=headers)

                        if student_home_response.status_code != 200:
                            print('Failed to redirect to Student Home Page')
                            return

                else:
                    print("Login request failed. Status code:", post_response.status_code)
            else:
                print("Failed to download CAPTCHA image. Status code:", captcha_response.status_code)
        else:
            print("CAPTCHA image not found on the page.")
    else:
        print("Failed to fetch login page. Status code:", response.status_code)


def get_marks(Sem):

    student_academics_url = "/Academics.aspx"
    full_academic_url = base_url + student_academics_url
    student_academic_response = requests.get(full_academic_url, headers=headers)

    if student_academic_response.status_code != 200:
            print('Redirected to Academics Page successfully')
            return

    with open('output.txt', 'w') as file:
        file.write(student_academic_response.text)

    soup = BeautifulSoup(student_academic_response.text, 'html.parser')
    table = soup.find('table', id='tblAttendancePercentage') ######
    print(table)


base_url = "https://slcm.manipal.edu"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'Cookie': 'ASP.NET_SessionId=junptmnf0ncgtk4wrx4qonpm',
    'Referer': 'https://slcm.manipal.edu',
}
username = 'Enter_your_SLCM_username'
password = 'Enter_your_SLCM_password'

login_with_captcha(base_url, headers, username, password)
get_marks(1)


In [ ]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
import pytesseract
import os

def login_with_captcha(base_url, headers, username, password):
    response = requests.get(base_url, headers=headers)

    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')

        captcha_img_tag = soup.find('img', id='imgCaptcha')

        if captcha_img_tag:
            captcha_img_url = base_url + "/" + captcha_img_tag['src']

            captcha_response = requests.get(captcha_img_url, headers=headers)

            if captcha_response.status_code == 200:
                with open('captcha_image.jpeg', 'wb') as f:
                    f.write(captcha_response.content)

                img = Image.open('captcha_image.jpeg')
                captcha_text = pytesseract.image_to_string(img)
                captcha_text = captcha_text.strip()

                os.remove('captcha_image.jpeg')

                form_data = {
                    'ScriptManager1': 'updpnl|btnLogin',
                    '__EVENTTARGET': '',
                    '__EVENTARGUMENT': '',
                    '__VIEWSTATE': soup.find('input', {'name': '__VIEWSTATE'})['value'],
                    '__VIEWSTATEGENERATOR': soup.find('input', {'name': '__VIEWSTATEGENERATOR'})['value'],
                    '__EVENTVALIDATION': soup.find('input', {'name': '__EVENTVALIDATION'})['value'],
                    'txtUserid': username,
                    'txtpassword': password,
                    'txtCaptcha': captcha_text,
                    '__ASYNCPOST': 'true',
                    'btnLogin': 'Sign in'
                }

                post_response = requests.post(base_url, data=form_data, headers=headers)

                if post_response.status_code == 200:
                    if 'Invalid Captcha' in post_response.text:
                        print('Captcha Error')
                    else:
                        student_home_url = "/studenthomepage.aspx"
                        full_student_home_url = base_url + student_home_url
                        student_home_response = requests.get(full_student_home_url, headers=headers)

                        if student_home_response.status_code != 200:
                            print('Failed to redirect to Student Home Page')
                            return

                else:
                    print("Login request failed. Status code:", post_response.status_code)
            else:
                print("Failed to download CAPTCHA image. Status code:", captcha_response.status_code)
        else:
            print("CAPTCHA image not found on the page.")
    else:
        print("Failed to fetch login page. Status code:", response.status_code)


def extract_attendance_details(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find('table', id='tblAttendancePercentage')

    for row in table.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')
        subject_code = columns[1].get_text().strip()
        subject = columns[2].get_text().strip()
        year_sem_trimester = columns[3].get_text().strip()
        total_class = columns[4].get_text().strip()
        days_present = columns[5].get_text().strip()
        days_absent = columns[6].get_text().strip()
        attendance_percentage = columns[7].get_text().strip()

        print(f"Subject Code: {subject_code}")
        print(f"Subject: {subject}")
        print(f"Year/Semester/Trimester: {year_sem_trimester}")
        print(f"Total Class: {total_class}")
        print(f"Days Present: {days_present}")
        print(f"Days Absent: {days_absent}")
        print(f"Attendance (%): {attendance_percentage}")
        print("----------------------------------------------------")


base_url = "https://slcm.manipal.edu"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'Cookie': 'ASP.NET_SessionId=junptmnf0ncgtk4wrx4qonpm',
    'Referer': 'https://slcm.manipal.edu',
}
username = 'Enter_your_SLCM_username'
password = 'Enter_your_SLCM_password'

login_with_captcha(base_url, headers, username, password)
student_academics_url = "/Academics.aspx"
full_academic_url = base_url + student_academics_url
student_academic_response = requests.get(full_academic_url, headers=headers)

if student_academic_response.status_code != 200:
        print('Redirected to Academics Page successfully')
else:
    extract_attendance_details(student_academic_response.text)


In [ ]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
import pytesseract
import os

def login_with_captcha(base_url, headers, username, password):
    response = requests.get(base_url, headers=headers)

    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')

        captcha_img_tag = soup.find('img', id='imgCaptcha')

        if captcha_img_tag:
            captcha_img_url = base_url + "/" + captcha_img_tag['src']

            captcha_response = requests.get(captcha_img_url, headers=headers)

            if captcha_response.status_code == 200:
                with open('captcha_image.jpeg', 'wb') as f:
                    f.write(captcha_response.content)

                img = Image.open('captcha_image.jpeg')
                captcha_text = pytesseract.image_to_string(img)
                captcha_text = captcha_text.strip()

                os.remove('captcha_image.jpeg')

                form_data = {
                    'ScriptManager1': 'updpnl|btnLogin',
                    '__EVENTTARGET': '',
                    '__EVENTARGUMENT': '',
                    '__VIEWSTATE': soup.find('input', {'name': '__VIEWSTATE'})['value'],
                    '__VIEWSTATEGENERATOR': soup.find('input', {'name': '__VIEWSTATEGENERATOR'})['value'],
                    '__EVENTVALIDATION': soup.find('input', {'name': '__EVENTVALIDATION'})['value'],
                    'txtUserid': username,
                    'txtpassword': password,
                    'txtCaptcha': captcha_text,
                    '__ASYNCPOST': 'true',
                    'btnLogin': 'Sign in'
                }

                post_response = requests.post(base_url, data=form_data, headers=headers)

                if post_response.status_code == 200:
                    if 'Invalid Captcha' in post_response.text:
                        print('Captcha Error')
                    else:
                        student_home_url = "/studenthomepage.aspx"
                        full_student_home_url = base_url + student_home_url
                        student_home_response = requests.get(full_student_home_url, headers=headers)

                        if student_home_response.status_code != 200:
                            print('Failed to redirect to Student Home Page')
                            return

                else:
                    print("Login request failed. Status code:", post_response.status_code)
            else:
                print("Failed to download CAPTCHA image. Status code:", captcha_response.status_code)
        else:
            print("CAPTCHA image not found on the page.")
    else:
        print("Failed to fetch login page. Status code:", response.status_code)


def extract_attendance_details(html_content, output_file):
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find('table', id='tblAttendancePercentage')

    with open(output_file, 'w') as file:
        for row in table.find_all('tr')[1:]:  # Skip the header row
            columns = row.find_all('td')
            subject_code = columns[1].get_text().strip()
            subject = columns[2].get_text().strip()
            year_sem_trimester = columns[3].get_text().strip()
            total_class = columns[4].get_text().strip()
            days_present = columns[5].get_text().strip()
            days_absent = columns[6].get_text().strip()
            attendance_percentage = columns[7].get_text().strip()

            file.write(f"Subject Code: {subject_code}\n")
            file.write(f"Subject: {subject}\n")
            file.write(f"Year/Semester/Trimester: {year_sem_trimester}\n")
            file.write(f"Total Class: {total_class}\n")
            file.write(f"Days Present: {days_present}\n")
            file.write(f"Days Absent: {days_absent}\n")
            file.write(f"Attendance (%): {attendance_percentage}\n")
            file.write("----------------------------------------------------\n")


base_url = "https://slcm.manipal.edu"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'Cookie': 'ASP.NET_SessionId=junptmnf0ncgtk4wrx4qonpm',
    'Referer': 'https://slcm.manipal.edu',
}
username = 'Enter_your_SLCM_username'
password = 'Enter_your_SLCM_password'
output_file = 'Attendance_6_sem.txt'

login_with_captcha(base_url, headers, username, password)
student_academics_url = "/Academics.aspx"
full_academic_url = base_url + student_academics_url
student_academic_response = requests.get(full_academic_url, headers=headers)

if student_academic_response.status_code != 200:
    print('Redirected to Academics Page successfully')
else:
    extract_attendance_details(student_academic_response.text, output_file)


In [ ]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
import pytesseract

import os
def login_with_captcha(base_url, headers, username, password):
    response = requests.get(base_url, headers=headers)

    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')

        captcha_img_tag = soup.find('img', id='imgCaptcha')

        if captcha_img_tag:
            captcha_img_url = base_url + "/" + captcha_img_tag['src']

            captcha_response = requests.get(captcha_img_url, headers=headers)

            if captcha_response.status_code == 200:
                with open('captcha_image.jpeg', 'wb') as f:
                    f.write(captcha_response.content)

                img = Image.open('captcha_image.jpeg')
                captcha_text = pytesseract.image_to_string(img)
                captcha_text = captcha_text.strip()

                os.remove('captcha_image.jpeg')

                form_data = {
                    'ScriptManager1': 'updpnl|btnLogin',
                    '__EVENTTARGET': '',
                    '__EVENTARGUMENT': '',
                    '__VIEWSTATE': soup.find('input', {'name': '__VIEWSTATE'})['value'],
                    '__VIEWSTATEGENERATOR': soup.find('input', {'name': '__VIEWSTATEGENERATOR'})['value'],
                    '__EVENTVALIDATION': soup.find('input', {'name': '__EVENTVALIDATION'})['value'],
                    'txtUserid': username,
                    'txtpassword': password,
                    'txtCaptcha': captcha_text,
                    '__ASYNCPOST': 'true',
                    'btnLogin': 'Sign in'
                }

                post_response = requests.post(base_url, data=form_data, headers=headers)

                if post_response.status_code == 200:
                    if 'Invalid Captcha' in post_response.text:
                        print('Captcha Error')
                    else:
                        student_home_url = "/studenthomepage.aspx"
                        full_student_home_url = base_url + student_home_url
                        student_home_response = requests.get(full_student_home_url, headers=headers)

                        if student_home_response.status_code != 200:
                            print('Failed to redirect to Student Home Page')
                            return

                else:
                    print("Login request failed. Status code:", post_response.status_code)
            else:
                print("Failed to download CAPTCHA image. Status code:", captcha_response.status_code)
        else:
            print("CAPTCHA image not found on the page.")
    else:
        print("Failed to fetch login page. Status code:", response.status_code)


def extract_attendance_details(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find('table', id='tblAttendancePercentage')

    attendance_details = []
    for row in table.find_all('tr')[1:]:  # Skip the header row
        columns = row.find_all('td')
        subject_code = columns[1].get_text().strip()
        subject = columns[2].get_text().strip()
        year_sem_trimester = columns[3].get_text().strip()
        total_class = columns[4].get_text().strip()
        days_present = columns[5].get_text().strip()
        days_absent = columns[6].get_text().strip()
        attendance_percentage = columns[7].get_text().strip()

        attendance_details.append({
            "Subject Code": subject_code,
            "Subject": subject,
            "Year/Semester/Trimester": year_sem_trimester,
            "Total Class": total_class,
            "Days Present": days_present,
            "Days Absent": days_absent,
            "Attendance (%)": attendance_percentage
        })

    return attendance_details


base_url = "https://slcm.manipal.edu"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'Cookie': 'ASP.NET_SessionId=junptmnf0ncgtk4wrx4qonpm',
    'Referer': 'https://slcm.manipal.edu',
}
username = 'Enter_your_SLCM_username'
password = 'Enter_your_SLCM_password'

login_with_captcha(base_url, headers, username, password)
student_academics_url = "/Academics.aspx"
full_academic_url = base_url + student_academics_url
student_academic_response = requests.get(full_academic_url, headers=headers)

if student_academic_response.status_code != 200:
    print('Redirected to Academics Page successfully')
else:
    attendance_details = extract_attendance_details(student_academic_response.text)
    with open('Attendance_6_sem.json', 'w') as file:
        json.dump(attendance_details, file, indent=4)


In [ ]:
import requests
from bs4 import BeautifulSoup
import json

def login_with_captcha(base_url, headers, username, password):
    response = requests.get(base_url, headers=headers)

    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')

        captcha_img_tag = soup.find('img', id='imgCaptcha')

        if captcha_img_tag:
            captcha_img_url = base_url + "/" + captcha_img_tag['src']

            captcha_response = requests.get(captcha_img_url, headers=headers)

            if captcha_response.status_code == 200:
                with open('captcha_image.jpeg', 'wb') as f:
                    f.write(captcha_response.content)

                img = Image.open('captcha_image.jpeg')
                captcha_text = pytesseract.image_to_string(img)
                captcha_text = captcha_text.strip()

                os.remove('captcha_image.jpeg')

                form_data = {
                    'ScriptManager1': 'updpnl|btnLogin',
                    '__EVENTTARGET': '',
                    '__EVENTARGUMENT': '',
                    '__VIEWSTATE': soup.find('input', {'name': '__VIEWSTATE'})['value'],
                    '__VIEWSTATEGENERATOR': soup.find('input', {'name': '__VIEWSTATEGENERATOR'})['value'],
                    '__EVENTVALIDATION': soup.find('input', {'name': '__EVENTVALIDATION'})['value'],
                    'txtUserid': username,
                    'txtpassword': password,
                    'txtCaptcha': captcha_text,
                    '__ASYNCPOST': 'true',
                    'btnLogin': 'Sign in'
                }

                post_response = requests.post(base_url, data=form_data, headers=headers)

                if post_response.status_code == 200:
                    if 'Invalid Captcha' in post_response.text:
                        print('Captcha Error')
                    else:

                        student_home_url = "/studenthomepage.aspx"
                        full_student_home_url = base_url + student_home_url
                        student_home_response = requests.get(full_student_home_url, headers=headers)

                        if student_home_response.status_code != 200:
                            print('Failed to redirect to Student Home Page')
                            return

                else:
                    print("Login request failed. Status code:", post_response.status_code)
            else:
                print("Failed to download CAPTCHA image. Status code:", captcha_response.status_code)
        else:
            print("CAPTCHA image not found on the page.")
    else:
        print("Failed to fetch login page. Status code:", response.status_code)


def get_marks(Sem):

    student_academics_url = "/Academics.aspx"
    full_academic_url = base_url + student_academics_url
    student_academic_response = requests.get(full_academic_url, headers=headers)

    if student_academic_response.status_code != 200:
            print('Redirected to Academics Page successfully')
            return

    with open('output.txt', 'w') as file:
        file.write(student_academic_response.text)

    soup = BeautifulSoup(student_academic_response.text, 'html.parser')
    div_panel_group = soup.find('div', class_='panel-group internalMarks')

    marks_data = []

    if div_panel_group:
        lines = str(div_panel_group).split('\n')
        subject_details = []
        for line in lines:
            if 'Subject Code' in line:
                subject_details.append(line.split('</b>')[1])

        for i in range(9):
            marks_text = subject_details[i] + "\n"
            marks_text += "--------------------------------\n"

            marks_obtained_id = f'ContentPlaceHolder1_RepeaterPrintInternal_lblTotal_{i}'
            marks_obtained_element = div_panel_group.find('span', id=marks_obtained_id)
            if marks_obtained_element:
                marks_obtained = marks_obtained_element.text.strip().split(':')[-1].strip()
                marks_text += "Marks Obtained: " + marks_obtained + "\n"
            else:
                marks_text += "Marks Obtained: NA\n"

            max_marks_id = f'ContentPlaceHolder1_RepeaterPrintInternal_Label1_{i}'
            max_marks_element = div_panel_group.find('span', id=max_marks_id)
            if max_marks_element:
                max_marks = max_marks_element.text.strip().split(':')[-1].strip()
                marks_text += "Max Marks: " + max_marks + "\n"
            else:
                marks_text += "Max Marks: NA\n"

            assignment_table_id = f'ContentPlaceHolder1_RepeaterPrintInternal_pnlAssignment_{i}'
            assignment_table_element = div_panel_group.find('div', id=assignment_table_id)

            if assignment_table_element:
                marks_text += "Assignment Details:\n"
                assignment_table = assignment_table_element.find('table')
                rows = assignment_table.find_all('tr')
                for row in rows[1:]:
                    columns = row.find_all('td')
                    assignment_name = columns[0].text.strip()
                    max_marks = columns[1].text.strip()
                    marks_obtained = columns[2].text.strip()
                    marks_text += f"Assignment: {assignment_name}, Max Marks: {max_marks}, Marks Obtained: {marks_obtained}\n"

            marks_text += "--------------------------------\n\n"
            marks_data.append(marks_text)

    else:
        print("Div element with class 'panel-group internalMarks' not found in the HTML content.")

    return marks_data


base_url = "https://slcm.manipal.edu"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'Cookie': 'ASP.NET_SessionId=junptmnf0ncgtk4wrx4qonpm',
    'Referer': 'https://slcm.manipal.edu',
}
username = 'Enter_your_SLCM_username'
password = 'Enter_your_SLCM_password'

login_with_captcha(base_url, headers, username, password)
marks_data = get_marks(1)

# Writing marks data to a JSON file
with open('Marks_6_sem.json', 'w') as file:
    json.dump(marks_data, file, indent=4)


In [ ]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
import pytesseract
import os


def login_with_captcha(base_url, headers, username, password):
    response = requests.get(base_url, headers=headers)

    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')

        captcha_img_tag = soup.find('img', id='imgCaptcha')

        if captcha_img_tag:
            captcha_img_url = base_url + "/" + captcha_img_tag['src']

            captcha_response = requests.get(captcha_img_url, headers=headers)

            if captcha_response.status_code == 200:
                with open('captcha_image.jpeg', 'wb') as f:
                    f.write(captcha_response.content)

                img = Image.open('captcha_image.jpeg')
                captcha_text = pytesseract.image_to_string(img)
                captcha_text = captcha_text.strip()

                os.remove('captcha_image.jpeg')

                form_data = {
                    'ScriptManager1': 'updpnl|btnLogin',
                    '__EVENTTARGET': '',
                    '__EVENTARGUMENT': '',
                    '__VIEWSTATE': soup.find('input', {'name': '__VIEWSTATE'})['value'],
                    '__VIEWSTATEGENERATOR': soup.find('input', {'name': '__VIEWSTATEGENERATOR'})['value'],
                    '__EVENTVALIDATION': soup.find('input', {'name': '__EVENTVALIDATION'})['value'],
                    'txtUserid': username,
                    'txtpassword': password,
                    'txtCaptcha': captcha_text,
                    '__ASYNCPOST': 'true',
                    'btnLogin': 'Sign in'
                }

                post_response = requests.post(base_url, data=form_data, headers=headers)

                if post_response.status_code == 200:
                    if 'Invalid Captcha' in post_response.text:
                        print('Captcha Error')
                    else:

                        student_home_url = "/studenthomepage.aspx"
                        full_student_home_url = base_url + student_home_url
                        student_home_response = requests.get(full_student_home_url, headers=headers)

                        if student_home_response.status_code != 200:
                            print('Failed to redirect to Student Home Page')
                            return

                else:
                    print("Login request failed. Status code:", post_response.status_code)
            else:
                print("Failed to download CAPTCHA image. Status code:", captcha_response.status_code)
        else:
            print("CAPTCHA image not found on the page.")
    else:
        print("Failed to fetch login page. Status code:", response.status_code)

def get_grades():
    student_academics_url = "/GradeSheet.aspx"
    full_academic_url = base_url + student_academics_url
    student_academic_response = requests.get(full_academic_url, headers=headers)

    if student_academic_response.status_code != 200:
        print('Redirected to Academics Page successfully')
        return

    with open('output.txt', 'w') as file:
        file.write(student_academic_response.text)

    soup = BeautifulSoup(student_academic_response.text, 'html.parser')
    table = soup.find('table', id='ContentPlaceHolder1_grvGradeSheet')

    if table:
        rows = table.find_all('tr')

        grades_data = []

        for row in rows[1:]:
            cells = row.find_all('td')
            sl_no = cells[0].text.strip()
            subject_code = cells[1].text.strip()
            subject_name = cells[2].text.strip()
            semester = cells[3].text.strip()
            grade = cells[4].text.strip() if cells[4].text.strip() else 'NA'
            credit = cells[5].text.strip()

            # Create dictionary for each row
            row_data = {
                "Sl No.": sl_no,
                "Subject Code": subject_code,
                "Subject Name": subject_name,
                "Actual Semester/Year": semester,
                "Grade": grade,
                "Credit": credit
            }

            grades_data.append(row_data)

        # Write data to JSON file
        with open('grades_data.json', 'w') as json_file:
            json.dump(grades_data, json_file, indent=4)

        print("Grades data written to grades_data.json file.")

    else:
        print("Grade sheet table not found.")



base_url = "https://slcm.manipal.edu"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'Cookie': 'ASP.NET_SessionId=junptmnf0ncgtk4wrx4qonpm',
    'Referer': 'https://slcm.manipal.edu',
}
username = 'Enter_your_SLCM_username'
password = 'Enter_your_SLCM_password'

login_with_captcha(base_url, headers, username, password)
get_grades()



Grades data written to grades_data.json file.


In [ ]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
import pytesseract
import os


def login_with_captcha(base_url, headers, username, password):
    response = requests.get(base_url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        captcha_img_tag = soup.find('img', id='imgCaptcha')

        if captcha_img_tag:
            captcha_img_url = base_url + "/" + captcha_img_tag['src']

            captcha_response = requests.get(captcha_img_url, headers=headers)

            if captcha_response.status_code == 200:
                with open('captcha_image.jpeg', 'wb') as f:
                    f.write(captcha_response.content)

                img = Image.open('captcha_image.jpeg')
                captcha_text = pytesseract.image_to_string(img)
                captcha_text = captcha_text.strip()

                os.remove('captcha_image.jpeg')

                form_data = {
                    'ScriptManager1': 'updpnl|btnLogin',
                    '__EVENTTARGET': '',
                    '__EVENTARGUMENT': '',
                    '__VIEWSTATE': soup.find('input', {'name': '__VIEWSTATE'})['value'],
                    '__VIEWSTATEGENERATOR': soup.find('input', {'name': '__VIEWSTATEGENERATOR'})['value'],
                    '__EVENTVALIDATION': soup.find('input', {'name': '__EVENTVALIDATION'})['value'],
                    'txtUserid': username,
                    'txtpassword': password,
                    'txtCaptcha': captcha_text,
                    '__ASYNCPOST': 'true',
                    'btnLogin': 'Sign in'
                }

                post_response = requests.post(base_url, data=form_data, headers=headers)

                if post_response.status_code == 200:
                    if 'Invalid Captcha' in post_response.text:
                        print('Captcha Error')
                    elif 'System doesn\'t recognize password.' in post_response.text:
                        print('System doesn\'t recognize password. (-_-)')
                    else:
                        student_home_url = "/studenthomepage.aspx"
                        full_student_home_url = base_url + student_home_url
                        student_home_response = requests.get(full_student_home_url, headers=headers)

                        if student_home_response.status_code != 200:
                            print('Failed to redirect to Student Home Page')
                            return

                else:
                    print("Login request failed. Status code:", post_response.status_code)
            else:
                print("Failed to download CAPTCHA image. Status code:", captcha_response.status_code)
        else:
            print("CAPTCHA image not found on the page.")
    else:
        print("Failed to fetch login page. Status code:", response.status_code)


def get_gpa():
    student_academics_url = "/GradeSheet.aspx"
    full_academic_url = base_url + student_academics_url
    student_academic_response = requests.get(full_academic_url, headers=headers)

    if student_academic_response.status_code != 200:
        print('Redirected to Academics Page successfully')
        return

    with open('output.txt', 'w') as file:
        file.write(student_academic_response.text)

    soup = BeautifulSoup(student_academic_response.text, 'html.parser')
    gpa_div = soup.find('div', class_='col-xs-6 col-md-6')

    if gpa_div:
        gpa_span = gpa_div.find('span', id='ContentPlaceHolder1_lblGPA')
        if gpa_span:
            gpa = gpa_span.text.strip()
            print("GPA:", gpa)
        else:
            print("GPA not found.")
    else:
        print("GPA div not found.")


def get_cgpa():
    student_academics_url = "/GradeSheet.aspx"
    full_academic_url = base_url + student_academics_url
    student_academic_response = requests.get(full_academic_url, headers=headers)

    if student_academic_response.status_code != 200:
        print('Redirected to Academics Page successfully')
        return

    with open('output.txt', 'w') as file:
        file.write(student_academic_response.text)

    soup = BeautifulSoup(student_academic_response.text, 'html.parser')
    gpa_div = soup.find('div', class_='col-xs-6 col-md-6')

    if gpa_div:
        gpa_span = gpa_div.find('span', id='ContentPlaceHolder1_lblGPA')
        if gpa_span:
            gpa = gpa_span.text.strip()
            print("GPA:", gpa)
        else:
            print("GPA not found.")
    else:
        print("GPA div not found.")


def get_cgpa():
    student_academics_url = "/GradeSheet.aspx"
    full_academic_url = base_url + student_academics_url
    student_academic_response = requests.get(full_academic_url, headers=headers)

    if student_academic_response.status_code != 200:
        print('Redirected to Academics Page successfully')
        return

    with open('output.txt', 'w') as file:
        file.write(student_academic_response.text)

    soup = BeautifulSoup(student_academic_response.text, 'html.parser')
    cgpa_span = soup.find('span', id='ContentPlaceHolder1_lblCGPA')

    if cgpa_span:
        cgpa = cgpa_span.text.strip()
        print("CGPA:", cgpa)
    else:
        print("CGPA not found.")


def get_totalCreditsEarned():
    student_academics_url = "/GradeSheet.aspx"
    full_academic_url = base_url + student_academics_url
    student_academic_response = requests.get(full_academic_url, headers=headers)

    if student_academic_response.status_code != 200:
        print('Redirected to Academics Page successfully')
        return

    with open('output.txt', 'w') as file:
        file.write(student_academic_response.text)

    soup = BeautifulSoup(student_academic_response.text, 'html.parser')
    credits_span = soup.find('span',id="ContentPlaceHolder1_LabelTotalcredit")

    if credits_span:
        credits = credits_span.text.strip()
        print("total credits earned:", credits)
    else:
        print("credits not found.")

def get_grades():
    student_academics_url = "/GradeSheet.aspx"
    full_academic_url = base_url + student_academics_url
    student_academic_response = requests.get(full_academic_url, headers=headers)

    if student_academic_response.status_code != 200:
        print('Redirected to Academics Page successfully')
        return

    with open('output.txt', 'w') as file:
        file.write(student_academic_response.text)

    soup = BeautifulSoup(student_academic_response.text, 'html.parser')
    table = soup.find('table', id='ContentPlaceHolder1_grvGradeSheet')

    if table:
        rows = table.find_all('tr')

        grades_data = []

        for row in rows[1:]:
            cells = row.find_all('td')
            sl_no = cells[0].text.strip()
            subject_code = cells[1].text.strip()
            subject_name = cells[2].text.strip()
            semester = cells[3].text.strip()
            grade = cells[4].text.strip() if cells[4].text.strip() else 'NA'
            credit = cells[5].text.strip()

            # Create dictionary for each row
            row_data = {
                "Sl No.": sl_no,
                "Subject Code": subject_code,
                "Subject Name": subject_name,
                "Actual Semester/Year": semester,
                "Grade": grade,
                "Credit": credit
            }

            grades_data.append(row_data)

        # Write data to JSON file
        with open('grades_data.json', 'w') as json_file:
            json.dump(grades_data, json_file, indent=4)

        print("Grades data written to grades_data.json file.")

    else:
        print("Grade sheet table not found.")





base_url = "https://slcm.manipal.edu"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'Cookie': 'ASP.NET_SessionId=junptmnf0ncgtk4wrx4qonpm',
    'Referer': 'https://slcm.manipal.edu',
}
username = 'Enter_your_SLCM_username'
password = 'Dachu@2004'

login_with_captcha(base_url, headers, username, password)
get_gpa()
get_cgpa()
get_totalCreditsEarned()


System doesn't recognize password. (-_-)
Redirected to Academics Page successfully
Redirected to Academics Page successfully
Redirected to Academics Page successfully


In [ ]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
import pytesseract
import os
import json

def login_with_captcha(base_url, headers, username, password):
    response = requests.get(base_url, headers=headers)

    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')

        captcha_img_tag = soup.find('img', id='imgCaptcha')

        if captcha_img_tag:
            captcha_img_url = base_url + "/" + captcha_img_tag['src']

            captcha_response = requests.get(captcha_img_url, headers=headers)

            if captcha_response.status_code == 200:
                with open('captcha_image.jpeg', 'wb') as f:
                    f.write(captcha_response.content)

                img = Image.open('captcha_image.jpeg')
                captcha_text = pytesseract.image_to_string(img)
                captcha_text = captcha_text.strip()

                os.remove('captcha_image.jpeg')

                form_data = {
                    'ScriptManager1': 'updpnl|btnLogin',
                    '__EVENTTARGET': '',
                    '__EVENTARGUMENT': '',
                    '__VIEWSTATE': soup.find('input', {'name': '__VIEWSTATE'})['value'],
                    '__VIEWSTATEGENERATOR': soup.find('input', {'name': '__VIEWSTATEGENERATOR'})['value'],
                    '__EVENTVALIDATION': soup.find('input', {'name': '__EVENTVALIDATION'})['value'],
                    'txtUserid': username,
                    'txtpassword': password,
                    'txtCaptcha': captcha_text,
                    '__ASYNCPOST': 'true',
                    'btnLogin': 'Sign in'
                }

                post_response = requests.post(base_url, data=form_data, headers=headers)

                if post_response.status_code == 200:
                    if 'Invalid Captcha' in post_response.text:
                        print('Captcha Error')
                    else:

                        student_home_url = "/studenthomepage.aspx"
                        full_student_home_url = base_url + student_home_url
                        student_home_response = requests.get(full_student_home_url, headers=headers)

                        if student_home_response.status_code != 200:
                            print('Failed to redirect to Student Home Page')
                            return

                else:
                    print("Login request failed. Status code:", post_response.status_code)
            else:
                print("Failed to download CAPTCHA image. Status code:", captcha_response.status_code)
        else:
            print("CAPTCHA image not found on the page.")
    else:
        print("Failed to fetch login page. Status code:", response.status_code)

def get_gpa(base_url, headers):
    student_academics_url = "/GradeSheet.aspx"
    full_academic_url = base_url + student_academics_url
    student_academic_response = requests.get(full_academic_url, headers=headers)

    if student_academic_response.status_code != 200:
        print('Redirected to Academics Page successfully')
        return None

    with open('output.txt', 'w') as file:
        file.write(student_academic_response.text)

    soup = BeautifulSoup(student_academic_response.text, 'html.parser')
    gpa_div = soup.find('div', class_='col-xs-6 col-md-6')

    if gpa_div:
        gpa_span = gpa_div.find('span', id='ContentPlaceHolder1_lblGPA')
        if gpa_span:
            gpa = gpa_span.text.strip()
            print("GPA:", gpa)
            return gpa
        else:
            print("GPA not found.")
            return None
    else:
        print("GPA div not found.")
        return None

def get_cgpa(base_url, headers):
    student_academics_url = "/GradeSheet.aspx"
    full_academic_url = base_url + student_academics_url
    student_academic_response = requests.get(full_academic_url, headers=headers)

    if student_academic_response.status_code != 200:
        print('Redirected to Academics Page successfully')
        return None

    with open('output.txt', 'w') as file:
        file.write(student_academic_response.text)

    soup = BeautifulSoup(student_academic_response.text, 'html.parser')
    cgpa_span = soup.find('span', id='ContentPlaceHolder1_lblCGPA')

    if cgpa_span:
        cgpa = cgpa_span.text.strip()
        print("CGPA:", cgpa)
        return cgpa
    else:
        print("CGPA not found.")
        return None

def get_totalCreditsEarned(base_url, headers):
    student_academics_url = "/GradeSheet.aspx"
    full_academic_url = base_url + student_academics_url
    student_academic_response = requests.get(full_academic_url, headers=headers)

    if student_academic_response.status_code != 200:
        print('Redirected to Academics Page successfully')
        return None

    with open('output.txt', 'w') as file:
        file.write(student_academic_response.text)

    soup = BeautifulSoup(student_academic_response.text, 'html.parser')
    credits_span = soup.find('span',id="ContentPlaceHolder1_LabelTotalcredit")

    if credits_span:
        credits = credits_span.text.strip()
        print("total credits earned:", credits)
        return credits
    else:
        print("credits not found.")
        return None

def write_to_json(gpa, cgpa, total_credits_earned):
    student_data = {
        "GPA": gpa,
        "CGPA": cgpa,
        "Total Credits Earned": total_credits_earned
    }

    with open('gpa_cgpa_totalCreditsEarned.json', 'w') as json_file:
        json.dump(student_data, json_file, indent=4)

    print("Student data written to gpa_cgpa_totalCreditsEarned.json file.")

def scrape_student_data(base_url, headers, username, password):
    login_with_captcha(base_url, headers, username, password)
    gpa = get_gpa(base_url, headers)
    cgpa = get_cgpa(base_url, headers)
    total_credits_earned = get_totalCreditsEarned(base_url, headers)
    write_to_json(gpa, cgpa, total_credits_earned)

base_url = "https://slcm.manipal.edu"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'Cookie': 'ASP.NET_SessionId=junptmnf0ncgtk4wrx4qonpm',
    'Referer': 'https://slcm.manipal.edu',
}
username = 'Enter_your_SLCM_username'
password = 'Enter_your_SLCM_password'

scrape_student_data(base_url, headers, username, password)


In [ ]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
import pytesseract
import os
import json

def login_with_captcha(base_url, headers, username, password):
    response = requests.get(base_url, headers=headers)

    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')

        captcha_img_tag = soup.find('img', id='imgCaptcha')

        if captcha_img_tag:
            captcha_img_url = base_url + "/" + captcha_img_tag['src']

            captcha_response = requests.get(captcha_img_url, headers=headers)

            if captcha_response.status_code == 200:
                with open('captcha_image.jpeg', 'wb') as f:
                    f.write(captcha_response.content)

                img = Image.open('captcha_image.jpeg')
                captcha_text = pytesseract.image_to_string(img)
                captcha_text = captcha_text.strip()

                os.remove('captcha_image.jpeg')

                form_data = {
                    'ScriptManager1': 'updpnl|btnLogin',
                    '__EVENTTARGET': '',
                    '__EVENTARGUMENT': '',
                    '__VIEWSTATE': soup.find('input', {'name': '__VIEWSTATE'})['value'],
                    '__VIEWSTATEGENERATOR': soup.find('input', {'name': '__VIEWSTATEGENERATOR'})['value'],
                    '__EVENTVALIDATION': soup.find('input', {'name': '__EVENTVALIDATION'})['value'],
                    'txtUserid': username,
                    'txtpassword': password,
                    'txtCaptcha': captcha_text,
                    '__ASYNCPOST': 'true',
                    'btnLogin': 'Sign in'
                }

                post_response = requests.post(base_url, data=form_data, headers=headers)

                if post_response.status_code == 200:
                    if 'Invalid Captcha' in post_response.text:
                        print('Captcha Error')
                    else:

                        student_home_url = "/studenthomepage.aspx"
                        full_student_home_url = base_url + student_home_url
                        student_home_response = requests.get(full_student_home_url, headers=headers)

                        if student_home_response.status_code != 200:
                            print('Failed to redirect to Student Home Page')
                            return

                else:
                    print("Login request failed. Status code:", post_response.status_code)
            else:
                print("Failed to download CAPTCHA image. Status code:", captcha_response.status_code)
        else:
            print("CAPTCHA image not found on the page.")
    else:
        print("Failed to fetch login page. Status code:", response.status_code)

def get_gpa(base_url, headers):
    student_academics_url = "/GradeSheet.aspx"
    full_academic_url = base_url + student_academics_url
    student_academic_response = requests.get(full_academic_url, headers=headers)

    if student_academic_response.status_code != 200:
        print('Redirected to Academics Page successfully')
        return None

    with open('output.txt', 'w') as file:
        file.write(student_academic_response.text)

    soup = BeautifulSoup(student_academic_response.text, 'html.parser')
    gpa_div = soup.find('div', class_='col-xs-6 col-md-6')

    if gpa_div:
        gpa_span = gpa_div.find('span', id='ContentPlaceHolder1_lblGPA')
        if gpa_span:
            gpa = gpa_span.text.strip()
            print("GPA:", gpa)
            return gpa
        else:
            print("GPA not found.")
            return None
    else:
        print("GPA div not found.")
        return None

def get_cgpa(base_url, headers):
    student_academics_url = "/GradeSheet.aspx"
    full_academic_url = base_url + student_academics_url
    student_academic_response = requests.get(full_academic_url, headers=headers)

    if student_academic_response.status_code != 200:
        print('Redirected to Academics Page successfully')
        return None

    with open('output.txt', 'w') as file:
        file.write(student_academic_response.text)

    soup = BeautifulSoup(student_academic_response.text, 'html.parser')
    cgpa_span = soup.find('span', id='ContentPlaceHolder1_lblCGPA')

    if cgpa_span:
        cgpa = cgpa_span.text.strip()
        print("CGPA:", cgpa)
        return cgpa
    else:
        print("CGPA not found.")
        return None

def get_totalCreditsEarned(base_url, headers):
    student_academics_url = "/GradeSheet.aspx"
    full_academic_url = base_url + student_academics_url
    student_academic_response = requests.get(full_academic_url, headers=headers)

    if student_academic_response.status_code != 200:
        print('Redirected to Academics Page successfully')
        return None

    with open('output.txt', 'w') as file:
        file.write(student_academic_response.text)

    soup = BeautifulSoup(student_academic_response.text, 'html.parser')
    credits_span = soup.find('span',id="ContentPlaceHolder1_LabelTotalcredit")

    if credits_span:
        credits = credits_span.text.strip()
        print("total credits earned:", credits)
        return credits
    else:
        print("credits not found.")
        return None

def get_grades(base_url, headers):
    student_academics_url = "/GradeSheet.aspx"
    full_academic_url = base_url + student_academics_url
    student_academic_response = requests.get(full_academic_url, headers=headers)

    if student_academic_response.status_code != 200:
        print('Redirected to Academics Page successfully')
        return

    with open('output.txt', 'w') as file:
        file.write(student_academic_response.text)

    soup = BeautifulSoup(student_academic_response.text, 'html.parser')
    table = soup.find('table', id='ContentPlaceHolder1_grvGradeSheet')

    if table:
        rows = table.find_all('tr')

        grades_data = []

        for row in rows[1:]:
            cells = row.find_all('td')
            sl_no = cells[0].text.strip()
            subject_code = cells[1].text.strip()
            subject_name = cells[2].text.strip()
            semester = cells[3].text.strip()
            grade = cells[4].text.strip() if cells[4].text.strip() else 'NA'
            credit = cells[5].text.strip()

            # Create dictionary for each row
            row_data = {
                "Sl No.": sl_no,
                "Subject Code": subject_code,
                "Subject Name": subject_name,
                "Actual Semester/Year": semester,
                "Grade": grade,
                "Credit": credit
            }

            grades_data.append(row_data)

        # Write data to JSON file
        with open('COMBINED_grade_sheet.json', 'w') as json_file:
            student_data = {
                "GPA": get_gpa(base_url, headers),
                "CGPA": get_cgpa(base_url, headers),
                "Total Credits Earned": get_totalCreditsEarned(base_url, headers),
                "Grades": grades_data
            }
            json.dump(student_data, json_file, indent=4)

        print("Student data written to gpa_cgpa_totalCreditsEarned_grades.json file.")

    else:
        print("Grade sheet table not found.")

def scrape_student_data(base_url, headers, username, password):
    login_with_captcha(base_url, headers, username, password)
    get_grades(base_url, headers)

base_url = "https://slcm.manipal.edu"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'Cookie': 'ASP.NET_SessionId=junptmnf0ncgtk4wrx4qonpm',
    'Referer': 'https://slcm.manipal.edu',
}
username = 'Enter_your_SLCM_username'
password = 'Enter_your_SLCM_password'

scrape_student_data(base_url, headers, username, password)


In [ ]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
import pytesseract
import os

def login_with_captcha(base_url, headers, username, password):
    response = requests.get(base_url, headers=headers)

    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')

        captcha_img_tag = soup.find('img', id='imgCaptcha')

        if captcha_img_tag:
            captcha_img_url = base_url + "/" + captcha_img_tag['src']

            captcha_response = requests.get(captcha_img_url, headers=headers)

            if captcha_response.status_code == 200:
                with open('captcha_image.jpeg', 'wb') as f:
                    f.write(captcha_response.content)

                img = Image.open('captcha_image.jpeg')
                captcha_text = pytesseract.image_to_string(img)
                captcha_text = captcha_text.strip()

                os.remove('captcha_image.jpeg')

                form_data = {
                    'ScriptManager1': 'updpnl|btnLogin',
                    '__EVENTTARGET': '',
                    '__EVENTARGUMENT': '',
                    '__VIEWSTATE': soup.find('input', {'name': '__VIEWSTATE'})['value'],
                    '__VIEWSTATEGENERATOR': soup.find('input', {'name': '__VIEWSTATEGENERATOR'})['value'],
                    '__EVENTVALIDATION': soup.find('input', {'name': '__EVENTVALIDATION'})['value'],
                    'txtUserid': username,
                    'txtpassword': password,
                    'txtCaptcha': captcha_text,
                    '__ASYNCPOST': 'true',
                    'btnLogin': 'Sign in'
                }

                post_response = requests.post(base_url, data=form_data, headers=headers)

                if post_response.status_code == 200:
                    if 'Invalid Captcha' in post_response.text:
                        print('Captcha Error')
                    else:

                        student_home_url = "/studenthomepage.aspx"
                        full_student_home_url = base_url + student_home_url
                        student_home_response = requests.get(full_student_home_url, headers=headers)

                        if student_home_response.status_code != 200:
                            print('Failed to redirect to Student Home Page')
                            return

                else:
                    print("Login request failed. Status code:", post_response.status_code)
            else:
                print("Failed to download CAPTCHA image. Status code:", captcha_response.status_code)
        else:
            print("CAPTCHA image not found on the page.")
    else:
        print("Failed to fetch login page. Status code:", response.status_code)


def get_marks(Sem):

    student_academics_url = "/StudentProfile.aspx"
    full_academic_url = base_url + student_academics_url
    student_academic_response = requests.get(full_academic_url, headers=headers)

    if student_academic_response.status_code != 200:
            print('Redirected to Academics Page successfully')
            return

    with open('output.txt', 'w') as file:
        file.write(student_academic_response.text)

    soup = BeautifulSoup(student_academic_response.text, 'html.parser')
    table = soup.find('div', class_="tab-pane active")
    print(table)


base_url = "https://slcm.manipal.edu"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'Cookie': 'ASP.NET_SessionId=junptmnf0ncgtk4wrx4qonpm',
    'Referer': 'https://slcm.manipal.edu',
}
username = 'Enter_your_SLCM_username'
password = 'Enter_your_SLCM_password'

login_with_captcha(base_url, headers, username, password)
get_marks(1)

In [ ]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
import pytesseract
import os

def login_with_captcha(base_url, headers, username, password):
    response = requests.get(base_url, headers=headers)

    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')

        captcha_img_tag = soup.find('img', id='imgCaptcha')

        if captcha_img_tag:
            captcha_img_url = base_url + "/" + captcha_img_tag['src']

            captcha_response = requests.get(captcha_img_url, headers=headers)

            if captcha_response.status_code == 200:
                with open('captcha_image.jpeg', 'wb') as f:
                    f.write(captcha_response.content)

                img = Image.open('captcha_image.jpeg')
                captcha_text = pytesseract.image_to_string(img)
                captcha_text = captcha_text.strip()

                os.remove('captcha_image.jpeg')

                form_data = {
                    'ScriptManager1': 'updpnl|btnLogin',
                    '__EVENTTARGET': '',
                    '__EVENTARGUMENT': '',
                    '__VIEWSTATE': soup.find('input', {'name': '__VIEWSTATE'})['value'],
                    '__VIEWSTATEGENERATOR': soup.find('input', {'name': '__VIEWSTATEGENERATOR'})['value'],
                    '__EVENTVALIDATION': soup.find('input', {'name': '__EVENTVALIDATION'})['value'],
                    'txtUserid': username,
                    'txtpassword': password,
                    'txtCaptcha': captcha_text,
                    '__ASYNCPOST': 'true',
                    'btnLogin': 'Sign in'
                }

                post_response = requests.post(base_url, data=form_data, headers=headers)

                if post_response.status_code == 200:
                    if 'Invalid Captcha' in post_response.text:
                        print('Captcha Error')
                    else:

                        student_home_url = "/studenthomepage.aspx"
                        full_student_home_url = base_url + student_home_url
                        student_home_response = requests.get(full_student_home_url, headers=headers)

                        if student_home_response.status_code != 200:
                            print('Failed to redirect to Student Home Page')
                            return

                else:
                    print("Login request failed. Status code:", post_response.status_code)
            else:
                print("Failed to download CAPTCHA image. Status code:", captcha_response.status_code)
        else:
            print("CAPTCHA image not found on the page.")
    else:
        print("Failed to fetch login page. Status code:", response.status_code)


def get_personal_details():
    student_academics_url = "/StudentProfile.aspx"
    full_academic_url = base_url + student_academics_url
    student_academic_response = requests.get(full_academic_url, headers=headers)

    if student_academic_response.status_code != 200:
        print('Failed to fetch Student Profile Page')
        return

    soup = BeautifulSoup(student_academic_response.text, 'html.parser')

    # Find input fields containing the required information
    reg_no_input = soup.find('input', id='ContentPlaceHolder1_txtApplicationNumber')
    student_name_input = soup.find('input', id='ContentPlaceHolder1_txtNameAs12MarkCard')
    branch_input = soup.find('input', id='ContentPlaceHolder1_txtProgramBranch')
    email_input = soup.find('input', id='ContentPlaceHolder1_txtStudentEmailID')
    app_no_input = soup.find('input', id='ContentPlaceHolder1_txtUniversityRollNumber')
    phone_no_input = soup.find('input', id='ContentPlaceHolder1_txtStudentMobileNumberPresent')
    gender_input = soup.find('input', id='ContentPlaceHolder1_txtGender')
    dob_input = soup.find('input', id='ContentPlaceHolder1_txtDOB')

    # Extract values from input fields
    reg_no = reg_no_input['value'] if reg_no_input else 'N/A'
    student_name = student_name_input['value'] if student_name_input else 'N/A'
    branch = branch_input['value'] if branch_input else 'N/A'
    email = email_input['value'] if email_input else 'N/A'
    app_no = app_no_input['value'] if app_no_input else 'N/A'
    phone_no = phone_no_input['value'] if phone_no_input else 'N/A'
    gender = gender_input['value'] if gender_input else 'N/A'
    dob = dob_input['value'] if dob_input else 'N/A'

    # Create dictionary for personal details
    personal_details = {
        "Registration Number": reg_no,
        "Student Name": student_name,
        "Branch": branch,
        "Email ID": email,
        "Application Number": app_no,
        "Phone Number": phone_no,
        "Gender": gender,
        "Date of Birth": dob
    }

    # Write personal details to JSON file
    with open('personal_details.json', 'w') as json_file:
        json.dump(personal_details, json_file, indent=4)




base_url = "https://slcm.manipal.edu"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',
    'Cookie': 'ASP.NET_SessionId=junptmnf0ncgtk4wrx4qonpm',
    'Referer': 'https://slcm.manipal.edu',
}
username = 'Enter_your_SLCM_username'
password = 'Enter_your_SLCM_password'

login_with_captcha(base_url, headers, username, password)
get_personal_details()